In [82]:
import pandas as pd

In [364]:
from tidyBom import  tidyBom
from tidyYarns import tidyYarns

# Load functions and set to local variables
tidy_Bom = tidyBom() 
tidy_Yarns = tidyYarns()
tidy_BomYarns = pd.merge(tidy_Bom, tidy_Yarns, on = 'yarn', how = 'outer')


In [350]:
millstyles = tidy_BomYarns.groupby('millstyle')
totalLbs = (millstyles.agg({'lbsperpiece' : 'sum',
							'yarn' : 'count'})
		    		  .rename(columns = {"lbsperpiece" : "totallbsperpiece",
                                         "yarn" : "num_yarns"}))

In [378]:
millstylesyarns = tidy_BomYarns.groupby(['millstyle','type'])
sumtype = (millstylesyarns.agg({'lbsperpiece': 'sum'})
						   .rename(columns = {"lbsperpiece" : "sumyarntype"}))

In [374]:
joined_lbsType = pd.merge(totalLbs, sumtype, left_index = True, right_index = True).reset_index()
joined_lbsType = joined_lbsType.assign(perc_present = joined_lbsType.sumyarntype / joined_lbsType.totallbsperpiece)

In [460]:
joined_lbsType.head()

,millstyle,type,totallbsperpiece,num_yarns,sumyarntype,perc_present
0,1001-22413-1,Spun,55.6,1,55.6,1.000000
1,1001-23131-1,Spun,55.2,3,55.2,1.000000
2,1001-23131-2,Dye,55.2,3,18.3,0.331522
3,1001-23131-2,Spun,55.2,3,36.9,0.668478
4,1001-23131-4,Spun,55.2,3,55.2,1.000000


In [478]:
millstylesyarns = joined_lbsType.groupby(['millstyle','type'])
added_perc = 

#joined_lbsType = joined_lbsType.assign(perc_spun = lambda x: joined_lbsType.perc_present if x == 'Spun' else 0)

joined_lbsType.type.apply(lambda x: 1 if x == 'Spun' else 0)

0    1
1    1
2    0
3    1
4    1
Name: type, dtype: int64

In [491]:
joined_lbsType['perc_spun'] =  if x == 'Spun' else 0 for x in joined_lbsType['type']]

SyntaxError: invalid syntax (<ipython-input-491-134b9d8ec00a>, line 1)

In [ ]:
joined_lbsType.loc[joined]

In [456]:
joined_lbsType.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
millstyle           5 non-null object
type                5 non-null object
totallbsperpiece    5 non-null float64
num_yarns           5 non-null int64
sumyarntype         5 non-null float64
perc_present        5 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 320.0+ bytes


In [488]:
joined_lbsType.iloc[0]['type']

'Spun'

In [486]:
joined_lbsType.head()

,millstyle,type,totallbsperpiece,num_yarns,sumyarntype,perc_present,perc_spun
0,1001-22413-1,Spun,55.6,1,55.6,1.000000,0 1.000000 1 1.000000 2 0.331522 3 ...
1,1001-23131-1,Spun,55.2,3,55.2,1.000000,0 1.000000 1 1.000000 2 0.331522 3 ...
2,1001-23131-2,Dye,55.2,3,18.3,0.331522,0
3,1001-23131-2,Spun,55.2,3,36.9,0.668478,0 1.000000 1 1.000000 2 0.331522 3 ...
4,1001-23131-4,Spun,55.2,3,55.2,1.000000,0 1.000000 1 1.000000 2 0.331522 3 ...


In [509]:
def joinedM365Stops():
	# Load libraries and functions
	import pandas as pandas
	from tidyM365 import tidyM365
	from tidyStops import tidyStops

	# Set returned dataframe to local variables
	tidy_M365 = tidyM365()
	tidy_stops = tidyStops()

	# Merge dataframes
	tidy_M365Stops = pd.merge(tidy_M365, tidy_stops, on = ['date', 'loom', 'range'], how = 'inner')

	# 
	groupby = tidy_M365Stops.groupby(['date', 'range', 'millstyle', 'loom'])
	joined_M365Stops = (groupby.agg({'ppm' : 'mean',
									'mpx' : 'sum',
									'yds' : 'sum',
									'combinedstops' : 'sum'})
							.rename(columns = {'ppm' : 'avgppm',
											'mpx' : 'totalpx',
											'yds' : 'totalyards',
											'combinedstops' : 'totalstops'}))
	joined_M365Stops = joined_M365Stops.assign(yardsperpx = joined_M365Stops['totalyards'] / joined_M365Stops['totalpx'])

	return joined_M365Stops.reset_index()


In [510]:
joined_M365Stops = joinedM365Stops()
joined_M365Stops.shape

(7279, 9)

In [508]:
joined_M365Stops.head()

,date,range,millstyle,loom,avgppm,totalpx,totalyards,totalstops,yardsperpx
0,2015-07-15,542,542A14742-3,18,350.0,484.0,561.0,164.0,1.159091
1,2015-07-15,542,542A18155-1,26,360.0,174.0,280.5,132.0,1.612069
2,2015-07-15,542,542A23589-7,19,360.0,524.0,561.0,102.0,1.070611
3,2015-07-15,542,542A23906-1,6,370.0,1062.0,841.5,396.0,0.792373
4,2015-07-15,641,641A21316-2,23,350.0,538.0,544.5,94.0,1.012082
